# Automatically building and testing your code: continuous integration

Once you have a test suite for your Python package (even if it is not complete yet), you will want to run it often to check that all tests continue to pass as your package evolves. Doing this automatically is the domain of *continuous integration* and for scientific projects it is most easily done using the free, online services that I discuss in this chapter. Like complete documentation and a comprehensive test suite before, continuous integration is an essential component of a software package that is used by more than a few people and/or receives contributions from outside users (e.g., through pull requests). Setting up continuous integration for your software package will end up saving you lots of time by finding issues with your code quickly and making it less likely that you merge a change that has unexpected, bad consequences.

## Why continuous integration?

Continuous integration (CI) is the practice of integration all changes into the "main" copy of a code base (here, a Python package) on a frequent basis ("continuously"). The "main" copy in our case is the GitHub repository of the package, that is the bassis of all clones and forks of the code (I assume throughout these notes that the only way the development version of the code is shared is through the GitHub site, even for versions used by the same developer). The main reason to perform continuous integration is to catch any unmergeable changes to the code made in different copies of the code quickly when they can typically be more easily resolved. Continuous integration checks both that the package builds successfully and that it passes the tests in the test suite and this is considered to be a successful integration. 

To perform continuous integration efficiently, both the build and test system need to be *automated*, that is, they should be able to be run without any human intervention. We have seen how to build a test suite that can be run with a simple ``pytest`` command in the [previous chapter](05-Tests.ipynb); I will discuss how to automate the build with specific examples below. Automating the building and testing of the code is important for taking any human decisions and mistakes out of the loop and for being able to perform the continuous-integration procedure *very often*.

What do we in practice mean by "continuous", because obviously we aren't running the build-and-test procedure all the time? Continuous qualitatively means that we run the build-and-test procedure every time the code or any of its dependencies change. In practice, it is easier to know when one's own code changes than when the code's dependencies change and we typically run the continuous-integration procedure upon every push of changes to GitHub. That is, we can make multiple commits and run the integration tests each time we push a set of commits to GitHub. Ideally, we would run the integration tests in conjunction with each commit, but since integration tests can be expensive and long-to-run, a compromise of running whenever we think the code is ready for a push to GitHub is good and it is easy to setup with automation services. This does mean that one should push changes to GitHub often, typically at least once a day, to make sure that the continuous integration procedure is run often. The code will also change in response to patches or new features submitted through a pull request and it is good practice to run the continuous-integration procedure *before* merging changes from a pull request. It is easy to set this up to be done automatically and, indeed, having continuous integration set up is essential to being able to merge pull requests for your package, because otherwise it is difficult to know that the proposed changes do not break some unexpected part of your code. One typically runs the continuous-integration procedure for changes to any branch, not just ``master``.

The way your code runs also changes when its dependencies change. While one could in principle set up a "continuous" check for whether dependencies have changed, in practice this is easiest to spot by running the continuous integration procedure on a fixed schedule in addition to any runs in response to pushes or merged pull requests. That way, you can ensure that the integration tests are run even when the code is going through a stretch of minor development. These fixed-schedule tests could be run daily or weekly, depending on how often you think dependencies might change and/or how quickly you think you need to catch this. One way in which the automated build-and-test of your code is useful is that it shows that there is a working version of your code and how to get it to work, which you can point people to who have issues with installing and running your code.

The advantages of continuous integration are many: you will find issues quickly, keep your development version in a working state and, thus, always have a fully-functional version of your code during development (that is, not just at releases), and by making use of automated tools running your integration tests on different types of machines you are able to easily make sure that your code runs on all systems that you support, not just your own. But there are disadvantages as well, the main one being that setting up and maintaining the continuous-integration system takes quite a bit of time and quite often issues that pop up during the integration procedure are due to the way the build-and-test procedure is set up (which can be quite complex for a larger Python package), rather than being due to a real bug in the package itself. For example, the way you install dependencies in the integration step might break and you then have to fix that to keep the continuous-integration procedure working, even though there is likely nothing wrong with your code. Overall, I think even this type of time is well spent, because it is important to know at all times that your code can be built (including its dependencies).

There are *many* services available to perform continuous integration of code, because continuous integration is a crucial aspect of all software and software-backed services, allowing bug fixes and updates to be rolled out quickly and often. Continuous integration of course goes far beyond Python packages and is used in the entire range of software, apps, and online services, where it is often combined with *continuous deployment* (CD, leading to the abbreviation CI/CD), the practice of rolling out bug fixes and updates as soon as they are made and pass the integration tests. The most popular CI/CD services are ``Travis CI``, ``Circle CI``, ``Jenkins``, ``appveyor``, and since recently ``GitHub Actions``. I will only discuss ``Travis CI``, ``appveyor``, and ``GitHub Actions`` below, but to a large degree they all work in the same way.

## Continuous integration with ``Travis CI``

``Travis CI`` is a continuous-integration service that is seamlessly integrated with GitHub and is free for open-source projects, generously providing you with multiple runners for your build-and-test integrations. The way ``Travis CI`` works is that you connect it to your GiHub account, give it access to your repository, and once configured it will automatically run your build and test suites every time you push to any branch and every time somebody opens a pull request. It will notify you when things go wrong (or when things go right! But that's not typically as interesting...) and can send the results from tests on to other services (e.g., those that parse and display your test coverage statistics).

To get started, go to [https://travis-ci.com/](https://travis-ci.com/) and sign up with your GitHub account. You then need to give ``Travis CI`` permission to access your account's information. Once you're back at ``travis-ci.com``, you can click on your profile picture to bring up a big green ``Activate`` button, press this to activate the GitHub Apps integration which ``Travis CI`` uses to start testing and deploying on ``Travis CI``. Once you have clicked this, you will see a list of your repositories, if you click on the repository that you want to add to ``Travis CI``, you will be brought to its page, which looks like this for the example package ``exampy`` at first

To start building and testing our package with ``Travis CI``, we have to add a ``.travis.yml`` configuration file to the top-level directory of our repository and push this to GitHub. This ``.travis.yml`` file will contain *all* of the information to configure the build-and-test procedure on ``Travis CI``. I will go over all of the parts in due course, so let's start with the simplest possible configuration for our ``exampy`` project, which looks like
```
language: python

python:
  - "3.7"

install:
  - python setup.py develop

script:
  - echo 0
```
This configuration file does only the most basic things: (i) it states what the language of the code (Python) and what version to use, (ii) it installs the code in the ``install:`` section, and (iii) it has a trivial statement in the ``script:`` section, which will later contain the running of the test suite. The reason that we need to add the trivial statement in the ``script:`` section is that without this section, ``Travis CI`` will label the run as failed. If you add this file to the GitHub repository and push it to GitHub, ``Travis CI`` is notified by GitHub that a change occurred in the repository, and ``Travis CI`` starts running the continuous integration procedure; the page on ``Travis CI`` will change to something like

(where the run should be \#1 for you, I messed up my own first run...). If you scroll down, you see the full log of what happened and you will see that ``Travis CI`` clones your repository, sets up Python, runs the ``install:`` section, followed by the ``script:`` section:

Next, we want to run our test suite with ``pytest`` and print the test coverage information using the ``pytest-cov`` plugin. To do this, we need to install ``pytest`` and the ``pytest-cov`` plugin in the ``install:`` section as well, and then run our test suite in the ``script:`` section, such that ``.travis.yml`` file now looks as follows
```
language: python

python:
  - "3.7"

install:
  - pip	install	pytest
  - pip	install	pytest-cov
  - python setup.py develop

script:
  - pytest -v tests/ --cov=exampy/
```
Pushing this change to GitHub, ``Travis CI`` automatically clones the updated repository and runs the build-and-test procedure. However, the tests fail, with the pertinent part of the log being

The tests failed, because we forgot to install ``scipy``, which is not a dependency of the ``exampy`` package itself (and, thus, we didn't list it in the ``install_requires`` section of the ``setup.py`` file), but it is a dependency of the test suite, because in one test we compare the ``exampy.integrate.simps`` procedure to numerical integration in ``scipy``. Thus, we need to also install ``scipy``, using the following ``.travis.yml``
```
language: python

python:
  - "3.7"

install:
  - pip	install	pytest
  - pip	install	pytest-cov
  - pip install scipy
  - python setup.py develop

script:
  - pytest -v tests/ --cov=exampy/
```
This time the tests pass without a hitch!

The ``.travis.yml`` configuration file has *lots* of possible sections to customize your build and test runs in any way that you want, which are documented in full [here](https://config.travis-ci.com/). When you are using the linux operating system (the default), you have access to a full Ubuntu distribution and you can run arbitrary commands in the sections ``install:``, ``before_install:`` (which contains commands to run before the installation of your code; technically it would have been better to install the test dependencies in the ``before_install:`` section, or in the ``before_script:`` section [see below] because they are not necessary for the code's installation itself), ``script:``, ``before_script:``, etc. There are also many other sections that allow you to install some dependencies more easily, to setup environment variables, to run your build-and-test integrations with different versions of dependencies (and Python itself), and there are sections ``after_success:`` and ``after_failure:`` to customize what ``Travis CI`` does upon successful or unsuccessful execution of your build-and-test run. In the remainder of this section, I will cover some of the most commonly-used customizations.

As a first example of a customization, let's say we want to explicitly specify the ``numpy`` version used by our code. To do this, we add an ``env:`` section that contains an environment variable ``NUMPY_VERSION`` that we set to the desired version. Then we can use this environment variable in the rest of the configuration file, e.g., as
```
language: python

python:
  - "3.7"

env:
  - NUMPY_VERSION=1.18

before_install:
  - pip install numpy==$NUMPY_VERSION

install:
  - python setup.py develop

before_script:
  - pip	install	pytest
  - pip	install	pytest-cov
  - pip install scipy

script:
  - pytest -v tests/ --cov=exampy/
```
where we have now explicitly included the ``numpy`` dependency install in the ``before_install:`` section (previously, we used the default ``numpy`` version available for this ``Travis CI`` disk image) and we have moved the installations that are only necessary for the test suite to the ``before_script:`` section. Pushing this change to GitHub, the log on ``Travis CI`` now shows that the requested version of ``numpy`` is installed, with the relevant part of the log being:

You can set multiple environment variables in the ``env:`` section, but note that all definitions should be part of a *single* dash. E.g., to also specify the ``scipy`` version, do
```
language: python

python:
  - "3.7"

env:
  - NUMPY_VERSION=1.18
    SCIPY_VERSION=1.4

before_install:
  - pip install numpy==$NUMPY_VERSION

install:
  - python setup.py develop

before_script:
  - pip	install	pytest
  - pip	install	pytest-cov
  - pip install scipy==$SCIPY_VERSION

script:
  - pytest -v tests/ --cov=exampy/
```
rather than
```
...
env:
  - NUMPY_VERSION=1.18
  - SCIPY_VERSION=1.4
...
```
because the latter would create two jobs, one with the ``numpy`` version set, the other with the ``scipy`` version set (see below). Another use of environment variables is to split your tests into sets to be executed in parallel, by explicitly setting the test files fed to ``pytest`` as an environment variable.

One of the great advantages of using a cloud-based continuous-integration system is that you can easily test your code with different versions of Python and your code's dependencies. ``Travis CI`` allows you to easily build *matrices* of jobs, by taking multiple values listed in some of the configuration sections. For example, if you list two major Python versions in the ``python:`` section and three minor ``numpy`` versions in the ``env:`` section as above, following the recommendations from a recent [proposal](https://numpy.org/neps/nep-0029-deprecation_policy.html) for which Python and ``numpy`` versions packages should support, ``Travis CI`` will run 6 different jobs, one for each combination of Python and ``numpy`` version. In the example that we have been considering, this gives the following ``.travis.yml``:
```
language: python

python:
  - "3.8"
  - "3.7"

env:
  - NUMPY_VERSION=1.18
  - NUMPY_VERSION=1.17
  - NUMPY_VERSION=1.16

before_install:
  - pip install numpy==$NUMPY_VERSION

install:
  - python setup.py develop

before_script:
  - pip	install	pytest
  - pip	install	pytest-cov
  - pip install scipy

script:
  - pytest -v tests/ --cov=exampy/
```
Pushing this to GitHub, we see that the ``Travis CI`` page for the current build-and-test integration looks as follows when the integration test is in progress:

Rather than seeing the log for a single job, we now see an overview of the six created jobs for the six combinations of Python and ``numpy`` versions. Some of these run in parallel. Clicking on one of the jobs will bring up the log for that particular job and ``Travis CI`` will report the status of each job as either a success or a failure; the entire combination only succeeds of all component jobs succeed (unless you [allow failures](https://docs.travis-ci.com/user/build-matrix/#rows-that-are-allowed-to-fail)). Once all jobs finish running, the final status looks as follows:

We see that one job failed! Another thing you notice when looking at the duration of each job is that the Python v3.8 / ``numpy`` v1.18 and all of the Python v3.7 finish in about 30 seconds, the other two Python v3.8 jobs run for about three minutes. Upon inspection of the logs, it becomes clear that the reason for this is that for these two jobs, no binary wheels for ``numpy`` are available; I will discuss binary wheels more in [Chapter 7](07-Package-Release.ipynb), but for now the important thing to know about them is that they allow you to install a package with ``pip`` without building it on your computer (which is what ``pip`` normally does). Thus, for these two versions, ``pip install numpy`` builds ``numpy`` from source and this takes a while. For the Pythonv3.8 / ``numpy`` v1.17 combination, this build actually fails on ``Travis CI``, which is the cause of this job's failure.

When your code includes a few dependencies that take a long (more than a dozen or so seconds) time to build from source (e.g., ``numpy`` *and* ``scipy``), it is beneficial to use the [Anaconda](https://www.anaconda.com/distribution/) Python distribution, which includes built versions of many packages that you might use. Because the entire Anaconda distribution is large and your ``Travis CI`` runs would have to download it every time, it is good to use the [Miniconda](https://docs.conda.io/en/latest/miniconda.html) version of Anaconda instead, which is a bare-bones version of Anaconda that comes with very few packages pre-installed to create a light-weight distribution. Using Miniconda requires us to add a few lines to the ``before_install:`` section to download Miniconda, set it up, and use it to install the dependencies. Note that we need to make sure that Miniconda gets set up for the same Python version that we requested in the ``python:`` section, by using the ``$TRAVIS_PYTHON_VERSION`` environment variable that ``Travis CI``` automatically defines. 

Re-writing the example above using Miniconda looks like
```
language: python

python:
  - "3.8"
  - "3.7"

env:
  - NUMPY_VERSION=1.18.1
  - NUMPY_VERSION=1.17,4
  - NUMPY_VERSION=1.16.6

before_install:
  - wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update conda
  - conda config --add channels conda-forge
  - conda create -n test-environment python=$TRAVIS_PYTHON_VERSION "numpy==$NUMPY_VERSION" scipy pip
  - source activate test-environment

install:
  - python setup.py develop

before_script:
  - pip	install	pytest
  - pip	install	pytest-cov

script:
  - pytest -v tests/ --cov=exampy/
```
Note that I have taken advantage of the fact that ``scipy`` can also be more easily installed using ``conda`` to simply include it in the main ``conda create`` command in the ``before_install:`` section. I also specified the exact version of ``numpy`` to use, because otherwise for, e.g., v1.17 ``conda`` tries to install v1.17.0 (which in this case fails, because it does not exist in Anaconda for Python 3.8). Finally, it's important to note that once you use conda, the Python version that is used is that given to the ``conda create`` command, which does not have to be the same as the one specified in the ``python:`` section. In particular, currently ``Travis CI`` does not contain Python v3.8 and it therefore has to download it first; you could simply use Python v3.7 in the ``python:`` section and then specify ``python=3.8`` in the ``conda create`` command to actually use Python v3.8 for the build and test, but in practice it's easier to keep the two Python versions consistent by using the ``$TRAVIS_PYTHON_VERSION`` environment variable.

This run now was successful, with the overview at the end looking like


Each job now takes about a minute to run, showing that using Miniconda significantly speeds up the run compared to building ``numpy`` from source (which above led to a > 2 minute job time; building ``scipy`` from source would more than double that).

At this point, you email inbox will be filled with emails from ``Travis CI`` telling you about the status of each integration run. You typically do not want to be updated about the status of every run, but simply when the status changes. For example, if a set of runs all end successfully, it's not that useful to get an email about this every time, but if a run suddenly fails, you will want to know that. Similarly, if your runs have been failing, it's useful to know when they start passing again (although that you will more likely be checking on the website directly). To only get notified upon a change in status from success to failure or failure to success, add a section to your ``.travis.yml`` file at the end that looks like
```
notifications:
  email:
    recipients:
      - YOUR_EMAIL
    on_success: change
    on_failure: change
```

Other useful parts of the ``.travis.yml`` file that I will not discuss in detail are:

* ``addons:`` allows you to specify dependencies that can be installed using, e.g., standard linux tools like ``apt-get``. For example, to use the GNU scientific library, use
```
addons:
  apt:
    packages:
      - libgsl0-dev
```

* If your tests make plots, you will run into errors, because the plots cannot be displayed. To avoid those, include the ``xvfb`` service that allows you to run graphical applications without a display:
```
services:
  - xvfb
```

* Building a matrix of jobs by multiplying options set in sections such as ``python:`` and ``env:`` quickly leads to large build matrices. If you just want to include an additional job with different parameters, you can include individual jobs in the ``matrix:`` section, e.g., include a single job that uses Python v3.6 and ``numpy`` v1.18.1 with
```
matrix:
  include:
    - python: "3.6"
      env: NUMPY_VERSION=1.18.1
```

  You can also [exclude](https://docs.travis-ci.com/user/build-matrix/#excluding-jobs) jobs from the matrix. Full documentation on the build matrix is given [here](https://docs.travis-ci.com/user/build-matrix/).
  
Finally, by default, ``Travis CI`` will run when you push a new commit or set of commits to GitHub (for any branch) or when someone opens or updates a pull request (in that case, ``Travis CI`` will automatically add status updates on whether the integration tests pass to the pull request's page). If in addition to this, you want the build of, say, your ``master`` branch to happen at least weekly whether or not a change occurred, you can do this with a "cron job" by going to your package's ``Travis CI`` page and navigating to the "Settings" under the "More Options" menu. There, there is a "Cron Jobs" part where you can choose a branch, how often to run (daily, weekly, or monthly currently), and whether to always run or run only if there hasn't been a commit in the last 24 hours (currently). This page looks as follows if you add the ``master`` branch on a weekly schedule, only running when there hasn't been a change in the last day:

I will discuss how you can process test coverage with ``Travis CI`` [below](06-Continuous-Integration.ipynb#Analyzing-test-coverage-online-using-codecov).

A final note: getting the installation of your dependencies, the build of your own package, and the test suite to run on ``Travis CI`` can be difficult and you will often end up with frustrating failed builds that are difficult to diagnose. In writing this section, I ran into syntax errors and unexpeced behavior multiple times. Unfortunately, there is nothing much you can do about this, because as far as I am aware one cannot replicate the ``Travis CI`` builds locally to test them before running them on ``Travis CI``. But there is lots of info in the online documentation and on the web to help you out and in the end it's wort the effort to know that your package is working as you expect at all times.

## Continuous integration for Windows: ``AppVeyor``

``Travis CI`` is great for testing your integration suite on a Linux operating system and it also has support for Mac OS X, although in practice running Python packages on Macs is so similar to running on Linux that there is little reason to test Mac OS X separately. But if you want to support Windows users, you will want to build and test your code on Windows, because there are subtle ways in which pure Python programs that you think are universal will fail on Windows (e.g., if you explicitly write paths as ``/path/to/file`` rather than using ``os.path.join``, which would properly put in the backslashes for Windows) and any compiled code will require significant modifications to run under Windows. While ``Travis CI`` has some support for Windows, only a limited set of features is available. However, ``AppVeyor`` is a continuous-integration system that is built around supporting Windows and it, like ``Travis CI``, allows free use for open-source projects (although under somewhat less generous terms: only a single runner is available at any time).

Overall [AppVeyor](https://www.appveyor.com/) works similar to ``Travis CI``. To start using it, sign in with your GitHub account, add your project by choosing among your GitHub repositories that will be displayed when you log in, and configure your build with a ``.appveyor.yml`` file (there is also a user interface, but it's easier to configure ``AppVeyor`` similar to ``Travis CI`` with a ``.yml`` file and you cannot both use the ``.appveyor.yml`` file and the user interface). The structure of the ``.appveyor.yml`` file is broadly similar to the ``.travis.yml`` file, with sections to set up the environment, the matrix of builds, the installation of dependencies and of the package itself, and a section to run the tests. ``AppVeyor`` builds run in the CMD shell or the [Windows PowerShell](https://docs.microsoft.com/en-us/powershell/scripting/getting-started/getting-started-with-windows-powershell), a shell that is similar in spirit to Unix-type shells, but quite different in many ways; I will use the CMD shell, because it is more similar to Unix-style shells. You can write arbitrary commands just as in the ``.travis.yml`` file, but they need to be able to run in the Powershell. This is a good time to point out that if you want to make your code work on Windows and set up the ``AppVeyor`` integration tests, it's helpful to have a Windows machine around to test things locally. If you don't have a Windows computer at hand, you can use a virtual machine, such as [VirtualBox](https://www.virtualbox.org/).

To illustrate the structure of the ``.appveyor.yml`` file, let's write the equivalent of the simple single-Python-and-``numpy`` that we got working in the previous section. This looks as follows
```
build: off

environment:
  PYTHON: "C:\\Python37"

install:
  - "set PATH=%PATH%;%PYTHON%\\Scripts"
  - "%PYTHON%\\python.exe -m pip install pytest"
  - "%PYTHON%\\python.exe -m pip install pytest-cov"
  - "%PYTHON%\\python.exe -m pip install scipy"
  - "%PYTHON%\\python.exe setup.py develop"

test_script:
  - pytest -v tests/ --cov=exampy/
```
Provided that we have added our package to our ``AppVeyor`` account, pushing this ``.appveyor.yml`` file to the package's GitHub repository will trigger a run of the ``AppVeyor`` integration test and we see the log in a similar way under the "Current build" as in ``Travis CI``:

What happens in this ``.appveyor.yml`` file is the following: (i) we turn ``build: off``, because ``build:`` is a MS-specific build process that we don't use for Python packages, (ii) we choose one of the [pre-installed Python](https://www.appveyor.com/docs/windows-images-software/#python) versions, (iii) we install dependencies and the package in a similar way as in ``.travis.yml``, with some Windows-specific tweaks, and (iv) we run the test script. Note that we need to add the ``C:\\Python37\\Scripts`` directory to the ``PATH``, because otherwise the ``pytest`` script is not found. Similar to ``Travis CI`` above, getting the ``AppVeyor`` integration runs to work can be a bit of trial and error and as you can see above, it took me six tries to get this simple example to work properly!

Note that you can use both CMD and PowerShell syntax in the same ``.appveyor.yml``, you simply need to prefix any PowerShell statements with ``ps:``, for example, you can do the path assignment in the PowerShell in the example above:
```
build: off

environment:
  PYTHON: "C:\\Python37"

install:
  - ps: $env:PATH="$env:PATH;$env:PYTHON\\Scripts"
  - "%PYTHON%\\python.exe -m pip install pytest"
  - "%PYTHON%\\python.exe -m pip install pytest-cov"
  - "%PYTHON%\\python.exe -m pip install scipy"
  - "%PYTHON%\\python.exe setup.py develop"

test_script:
  - pytest -v tests/ --cov=exampy/
```

Like with ``Travis CI`` above, you can define matrices of jobs to run and you can also use Miniconda to run Python, which is [natively installed](https://www.appveyor.com/docs/windows-images-software/#miniconda), so you don't have to download Miniconda like on ``Travis CI``. One limitation of the build matrices on ``AppVeyor`` is that you cannot automatically multiply different environment variables, but you have to write out all combinations by hand. So to combine two Python versions with three ``numpy`` versions as in the final ``Travis CI`` example above, we have to write out all six combinations. The equivalent of the ``Travis CI`` example with this setup that uses Miniconda for ``AppVeyor`` uses the following ``.appveyor.yml``
```
build: off

environment:
  MINICONDA: C:\\Miniconda37-x64

  matrix:
    - PYTHON_VERSION: 3.8
      NUMPY_VERSION: 1.18.1

    - PYTHON_VERSION: 3.8
      NUMPY_VERSION: 1.17.4

    - PYTHON_VERSION: 3.8
      NUMPY_VERSION: 1.16.6

    - PYTHON_VERSION: 3.7
      NUMPY_VERSION: 1.18.1

    - PYTHON_VERSION: 3.7
      NUMPY_VERSION: 1.17.4

    - PYTHON_VERSION: 3.7
      NUMPY_VERSION: 1.16.6

  
install:
  - cmd: call %MINICONDA%\Scripts\activate.bat
  - cmd: conda.exe update --yes --quiet conda
  - "set PATH=%MINICONDA%;%MINICONDA%\\Scripts;%PATH%"
  - conda config --set always_yes yes --set changeps1 no
  - conda info -a
  - "conda create -n test-environment python=%PYTHON_VERSION% numpy==%NUMPY_VERSION% scipy"
  - activate test-environment
  - python setup.py develop"

before_test:
  - pip install pytest
  - pip install pytest-cov
  
test_script:
  - pytest -v tests/ --cov=exampy/
```
Note that here I use the Miniconda for Python v3.7, even for the tests that use Python v3.8, because at the time of writing there is no direct support for Python v3.8 on ``Appveyor``. As you can see, the ``.appveyor`` file is quite similar to the ``.travis.yml`` file for this setup, except that we need to manually write out this job matrix.

Once we push this new ``.appveyor.yml`` file to GitHub, the various jobs start running and the "Current build" page changes to an overview of the different jobs:

Soon all of the individual jobs finish successfully and we can inspect their individual logs by clicking on them. For example, the final part of the Python v3.8 / ``numpy`` v1.17.4 looks like

As you can see, the tests pass and we are indeed using Python v3.8.

``AppVeyor`` by default will only email you upon a change in the status, so there is no need to configure that directly. Like ``Travis CI``, ``AppVeyor`` will run for any change to any branch and for any opened or updated pull request.

## ``GitHub Actions``, the new kid on the block

## Analyzing test coverage online using ``codecov``